<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/corso_data_science/machine_learning_pills/10_classificare_documenti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classificare Documenti col Deep Learning

<br>

![news](https://thedcpatriot.com/wp-content/uploads/2020/01/breaking.jpg)

<br>

[Image Credits](https://thedcpatriot.com/)

<br>

Questo notebook prende spunto da un [esercizio](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/3.6-classifying-newswires.ipynb) contenuto nel libro **Deep Learning with Python** di [Francois Chollet](https://fchollet.com/).

Sfrutta il dataset *Reuters* contenuto nella libreria keras di Tensorflow, e serve per classificare le notizie in 46 categorie a seconda del testo.

Nel nostro caso servirà a capire come trattare dati non tabulari, visto che abbiamo visto quasi solo esclusivamente questi ultimi durante le lezioni.

<br>

![Keras](https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png)

<br>

[Image Credits](https://keras.io/)

<br>

### Cos'è Keras?

[Keras](https://keras.io/) è una libreria open source per il machine learning e le reti neurali, scritta in Python. È progettata come un'interfaccia a un livello di astrazione superiore di altre librerie simili di più basso livello, e supporta come back-end le librerie [TensorFlow](https://www.tensorflow.org/), [Microsoft Cognitive Toolkit (CNTK)](https://docs.microsoft.com/en-us/cognitive-toolkit/) e [Theano](http://deeplearning.net/software/theano/). Progettata per permettere una **rapida prototipazione di deep neural network**, si concentra sulla **facilità d'uso, la modularità e l'estensibilità**. È stata sviluppata come parte del progetto di ricerca ONEIROS, e il suo autore principale è François Chollet, di Google. Nel 2017 il team di TensorFlow ha deciso di supportare Keras ufficialmente.

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

### Reuters Dataset

Il dataset Reuters è uno dei molti dataset già presenti in Keras. Contiene un set di notizie, e la loro classificazione, pubblicate da Reuters nel 1986. Ci sono 46 categorie ed è tra i dataset più utilizzati per gli esempi sui dati testuali.

Nel caricamento ci limiteremo ad un set di 10000 parole (le più frequenti), queste verranno codificate sotto forma di interi.

In [0]:
from tensorflow.keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [0]:
print("La lunghezza del train set è pari a: ", len(train_data))
print("La lunghezza del test set è pari a: ", len(test_data))


Vediamo come appare una notizia e la sua rispettiva classe di appartenenza.

In [0]:
print(train_data[0])
print(train_labels[0])

Per gli esseri umani Keras fornisce anche il dizionario per decodificare i messaggi e leggerli in forma testuale. Per le labels consultare [questo link](https://martin-thoma.com/nlp-reuters/), comunque la 3 è *money-fx*.

In [0]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
print(decoded_newswire)

### Preparazione del dato

#### Vectorize

Al momento ogni notizia è una sequenza di numeri di lunghezza diversa, noi vogliamo che ogni notizia sia un vettore di 10000 elementi che saranno:
* pari ad 1 se il termine è presente
* pari a 0 se il termine è assente

Alla fine avremo una matrice di N x 10000 elementi.

In [0]:
# Creiamo la funzione vectorize_sequences
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [0]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

print("dimensioni train: ", x_train.shape)
print("dimensioni test: ", x_test.shape)
print("\n","prime 5 righe train: ", "\n", x_train[:5])

#### One-Hot Encoding

<br>

![onehot](https://cdn.filestackcontent.com/DmN2NdYwSEyepOeMYn2V)

<br>

[Image Credits](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)

<br>

Il One-Hot encoding trasforma una variabile categoriale in N variabili numeriche binarie che indicano l'appartenenza o la non appartenenza ad una classe.

Nel nostro caso abbiamo 46 categorie quindi ogni label del diventerà un vettore di 46 elementi, 45 zero ed 1 uno.

In [0]:
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

print("one_hot_train_labels ", one_hot_train_labels.shape)
print("one_hot_test_labels ", one_hot_test_labels.shape)

print("\n","prime 5 labels: ", "\n",one_hot_train_labels[:5])

#### Costruzione del Validation Set

Per valutare la bontà del modello abbiamo bisogno di *isolare* dal training set un serto numero di osservazioni che andranno a far parte del set di validazione sul quele verrà validato il modello prima della previsione sul test set.

Il nostro validation set saranno le prime 1000 notizie.

Solitamente è sbagliato prendere le osservazioni in sequenza perchè il dataset potrebbe essere ordinato per classe (vedi Iris dataset), successivamente faremo anche il test estraendo il validation set in maniera casuale.

<br>

![validation](https://i.stack.imgur.com/pXAfX.png)

<br>

[Image Credits](https://datascience.stackexchange.com/questions/61467/clarification-on-train-test-and-val-and-how-to-use-implement-it)

In [0]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

print("x_val ", x_val.shape)
print("y_val ", y_val.shape)
print("\n")
print("partial_x_train ", partial_x_train.shape)
print("partial_y_train ", partial_y_train.shape)

### Costruzione del modello

<br>

![DNN](https://www.researchgate.net/profile/Charlotte_Pelletier/publication/331525817/figure/fig2/AS:733072932745216@1551789615161/Example-of-fully-connected-neural-network.png)

<br>

[Image Credits](https://www.researchgate.net/figure/Example-of-fully-connected-neural-network_fig2_331525817)

<br>

Il modello che andremo a costruire è una Dense Neural Network, il tipo di rete neurale "deep" più semplice, la nostra rete avrà:
* il livello di input che può prendere 10000 variabili alla volta (la numerosità di ogni osservazione nel train)
* due livelli nascosti con funzione di attivazione **ReLU**. Non esiste un metodo per scegliere quanti livelli e quanti nodi per livello inserire, è consigliato partire da architetture semplici *shallow network* e renderle più complesse ottimizzando la previsione.
* un livello di output con 46 nodi, uno per ogni classe, e funzione di attivazione **softmax** utilizzata per i classificatori multilabels.

Una volta creata l'architettura il modello và compilato e useremo:
* ottimizzatore **rmsprop** molto utilizzato nello studio delle sequenze
* come funzione da ottimizzare (loss) **categorical_crossentropy** visto che è un classificatore multilabel
* metrica **accuracy**

*N.B. Non approfondiremo i concetti come nodo, layer, rete, funzioni di attivazione, batch, ... per motivi di tempi e obiettivi non sono parte del corso.*

In [0]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10000,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(46, activation='softmax'))
model.summary()

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Ora parte l'allenamento del modello, scegliamo la numerosità dei batch e le epoche di allenamento.

In [0]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

#### Valutazione del modello

Per valutare le prestazioni confrontiamo il loss e l'accuratezza sul training set e test set.

In [0]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [0]:
results = model.evaluate(x_val, y_val)
print("_"*100)
print("Test Loss and Accuracy")
print("results ", results)

Questo è un chiaro caso di **overfitting**.
* La funzione di loss del train è più bassa della funzione di validation loss
* L'accuratezza sul train è più alta di quella sul validation

Il modello si adatta ai dati di train e non riesce a generalizzare. Proviamo a utilizzare un validation set estratto casualimente dal train set.

In [0]:
history = model.fit(x_train,
                    one_hot_train_labels,
                    epochs=20,
                    batch_size=512,
                    validation_split=0.1)

In [0]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

La situazione è addirittura peggiorata, quindi per cercare di avere un modello più performante che non ricada nell'overfitting cerchiamo di agire sull'architettura:
* aumentiamo i nodi per i livelli hidden
* inseriamo dei livelli di **Dropout**, questa tecnica **spegne** alcune connessioni tra i nodi costruendo modelli forse meno precisi ma molto più generalizzabili, prevenendo l'overfitting
* inseriamo la regolarizzazione *l1* che dà peso 0 alle variabili meno importanti così da fare una selezione delle stesse.

In [0]:
model = Sequential()
model.add(Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(10000,)))
model.add(Dropout(0.5))
model.add(Dense(256, kernel_regularizer=regularizers.l1(0.001), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(46, activation='softmax'))
model.summary()

In [0]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=256, validation_data=(x_val, y_val))

In [0]:
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Come si può vedere in questo caso il modello non và in overfitting e, seppur con fluttuazioni e rimanendo qualitativamente meno performante di quello di prima, ha ottime prestazioni sia sul train che sul validation set. Vediamo di seguito.

In [0]:
results = model.evaluate(x_val, y_val)
print("_"*100)
print("Test Loss and Accuracy")
print("results ", results)

**Nota. Al momento il nostro modello è quello uscito dall'ultima epoca di allenamento ma Keras ci fornisce un parametro per utilizzare il "migliore modello" uscito dalla fase di train.**

### Applicazione del modello sul test set

Applichiamo il nostro modello sul test set e facciamo i confronti col dato reale.

In [0]:
predictions = model.predict(x_test)

Nnotizia = 123

# Ogni previsione è un vettore di 46 elementi con la probabilità di appartenere ad una classe:
print(predictions[Nnotizia].shape)

# La somma dei coefficienti deve essere 1:
print(round(np.sum(predictions[Nnotizia])))

# Le classi con maggiore pobabilità di appartenenza:
print("Le classi con maggiore pobabilità di appartenenza: ", predictions[Nnotizia].argsort()[-3:][::-1])

# La classe con maggiore probabilità:
print("La classe prevista dal modello: ", np.argmax(predictions[Nnotizia]))
print("La calsse reale: ",test_labels[Nnotizia])

Creaiamo un vettore di classi per calcolare l'accuratezza sul test del nostro modello per:
* la Top 3
* il risultato esatto

In [0]:
# Creaiamo la top 3 matrix

Top3Preds = np.zeros((2246,3), dtype=int)
# print(Top3Preds.shape)

for SampleNum in range(predictions.shape[0]):
    Top3Preds[SampleNum] = predictions[SampleNum].argsort()[-3:][::-1]

# Creiamo la matrice di confusione

FinalPreds = np.zeros((2246,1), dtype=int)
# print(FinalPreds.shape)

for SampleNum in range(Top3Preds.shape[0]):
    if test_labels[SampleNum] in Top3Preds[SampleNum]:
        FinalPreds[SampleNum] = 1
        
FinalPreds = pd.DataFrame(FinalPreds)
NumTop3 = FinalPreds[0][FinalPreds[0] == 1].count()
percentTop3 = round(100 *NumTop3 / FinalPreds.shape[0], 1)

print('Percent of one from top 3 being correct ... ', percentTop3, '%')

In [0]:
TopPreds = np.zeros((2246,1), dtype=int)
for SampleNum in range(predictions.shape[0]):
  TopPreds[SampleNum] = np.argmax(predictions[SampleNum])

print('Accuracy Score :',round(accuracy_score(test_labels, TopPreds)*100,1),"%")

Quindi la previsone "secca" del nostro modello è al **69.1%** di accuratezza mentre quella sui primi 3 argomenti più probabili aumenta di 10 punti percentuali.

Ora se volete potete riprovare a:
* creare un modello con validation set scelto casualmente
* modificare le percentuali di dropout
* provare gli altri regularizers (l2, l1_l2)

## Approfondimenti

* [Regularization](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c)
* [Dropout](https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/)
* [Batch](https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network)
* [Documentazione di Keras sui layer](https://keras.io/layers/core/)
* [Notebook su Kaggle](https://www.kaggle.com/drscarlat/reuters-document-classification-with-keras-tf)
* [Confusion Matrix](https://www.geeksforgeeks.org/confusion-matrix-machine-learning/)